Nomes: Bel Cogo e Bruno da Siqueira Hoffmann.

Professor: Felipe de Morais.

Disciplina: Ciência de Dados e Big Data.

# Tarefa 5

O banco de dados utilizado para essa tarefa contém dados referentes a dados de transações, consistindo em 5 tabelas, sendo elas:

- sellers: Contém informações sobre os vendedores, incluindo o estado em que estão localizados.
- products: Detalha os produtos disponíveis no marketplace, incluindo categorias e pesos.
- orders: Fornece informações básicas sobre cada pedido, como registros de data/hora e contato do cliente.
- order_items: Agrupa os itens associados a cada pedido, incluindo detalhes de preço.
- order_reviews: Armazena as avaliações relacionadas a cada pedido, incluindo classificações numéricas e comentários em texto.

É possível verificar o ER do banco de dados abaixo:

![Diagrama ER.jpg](./Diagrama%20ER.jpg)

Para importar o banco de dados foi executado o comando abaixo:

In [2]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('transactional_data.sqlite')

## Pergunta 1

Pergunta: Quais são os sellers que possuem um faturamento inferior a R$ 100? E qual é o seu faturamento? Listando os 10 com menor faturamento e o seu estado.

Resposta:

In [30]:
query = """
  SELECT
    s.seller_id as identificador_vendedor,
    SUM(oi.price) as faturamento,
    s.seller_state as estado_do_vendedor
  FROM sellers s
    INNER JOIN order_items oi
      ON s.seller_id = oi.seller_id
  GROUP BY s.seller_id
  HAVING faturamento < 100
  ORDER BY faturamento ASC
  LIMIT 10;
"""

df = pd.read_sql(query, connection)
df

,identificador_vendedor,faturamento,estado_do_vendedor
0,77128dec4bec4878c37ab7d6169d6f26,6.50,SP
1,1fa2d3def6adfa70e58c276bb64fe5bb,6.90,SP
2,702835e4b785b67a084280efca355756,7.60,MG
3,ad14615bdd492b01b0d97922e87cb87f,8.25,SC
4,4965a7002cca77301c82d3f91b82e1a9,8.49,SP
5,0f94588695d71662beec8d883ffacf09,9.00,SC
6,c18309219e789960add0b2255ca4b091,9.90,RJ
7,95cca791657aabeff15a07eb152d7841,9.99,PR
8,344223b2a90784f64136a8a5da012e7f,10.90,SC
9,f9f68daa2af419d38f745478bcb3da1a,11.20,SP


## Questão 2

Pergunta: Quais os 5 produtos mais vendidos?

## Questão 3

Pergunta: Qual o estado com mais sellers? E qual é o valor total faturado?

## Questão 4

Pergunta: Qual seller tem mais pedidos com review ruim? E qual é a média de nota dele?

## Questão 5

Pergunta: Quais são os 10 produtos com mais ordens? E quanto ele fatura?

In [19]:
query = """
  SELECT 
    COUNT(1) as quantidade,
    p.product_id as identificador_produto, 
    p.product_category_name as nome_categoria,
    SUM(oi.price) as preco_total
  FROM products p 
    INNER JOIN order_items oi 
      on p.product_id = oi.product_id 
  GROUP BY oi.product_id 
  ORDER BY quantidade DESC
  LIMIT 10;
"""

df = pd.read_sql(query, connection)
df

,quantidade,identificador_produto,nome_categoria,preco_total
0,520,aca2eb7d00ea1a7b8ebd4e68314663af,moveis_decoracao,37104.30
1,484,422879e10f46682990de24d770e7f83d,ferramentas_jardim,26577.22
2,477,99a4788cb24856965c36a24e339b6058,cama_mesa_banho,42049.66
3,390,389d119b48cf3043d311335e499d9c6b,ferramentas_jardim,21336.79
4,388,368c6c730842d78016ad823897a372db,ferramentas_jardim,21056.80
5,373,53759a2ecddad2bb87a079a1f1519f73,ferramentas_jardim,20387.20
6,332,d1c427060a0f73f6b889a5c7c61f2ac4,informatica_acessorios,45620.56
7,321,53b36df67ebb7c41585e8d54d6772e08,relogios_presentes,37454.63
8,274,154e7e31ebfa092203795c972e5804a6,beleza_saude,6173.26
9,272,3dd2a17168ec895c781a9191c1e95ad7,informatica_acessorios,40782.80
